<a href="https://colab.research.google.com/github/itsinaam/Deep-Learning_projects/blob/main/Transfer_Learning_with_Data_Agumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:49<00:00, 24.0MB/s]
100% 1.06G/1.06G [00:49<00:00, 23.0MB/s]


In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('/content/dogs-vs-cats.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [ ]:
vgg = VGG16(
    weights='imagenet',
    input_shape = (150,150,3),
    include_top = False



)

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
model = Sequential()

model.add(vgg)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [ ]:
vgg.trainable= False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
batch_size = 32

train_degen = ImageDataGenerator(
       rescale = 1/255,
       shear_range = 0.2,
       zoom_range = 0.2,
       horizontal_flip=True
)

In [ ]:
test_degen = ImageDataGenerator(
        rescale = 1/255
)

In [ ]:
train_generator = train_degen.flow_from_directory(
        '/content/train',
        target_size = (150,150),
        batch_size = batch_size,
        class_mode = 'binary'


    
)

Found 20000 images belonging to 2 classes.


In [ ]:
validation_generator = train_degen.flow_from_directory(
        '/content/test',
        target_size = (150,150),
        batch_size = batch_size,
        class_mode = 'binary'


    
)

Found 5000 images belonging to 2 classes.


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit_generator(train_generator,epochs=10,validation_data=validation_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
610/625 [============================>.] - ETA: 1:54 - loss: 0.3076 - accuracy: 0.8685

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'],color='red',label='train_loss')
plt.plot(history.history['val_loss'],color='blue',label='test_loss')
plt.legend()
plt.show()


In [ ]:
plt.plot(history.history['accuracy'],color='red',label='train_accuracy')
plt.plot(history.history['val_accuracy'],color='blue',label='test_accuracy')
plt.legend()
plt.show()